In [19]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy
import random

from fastai.structured import *
from fastai.column_data import *

In [20]:
bs,bptt = 64,70

## Language modeling

### Data

In [21]:
import pandas as pd

In [22]:
PATH='data/high-flyers/'

df_tweets = pd.read_csv(f'{PATH}train.csv', index_col=0)

In [23]:
from IPython.display import HTML, display
display(df_tweets.head())

,text,airline,tweet_location,user_timezone,sentiment
id,,,,,
0,@JetBlue great flight! Great view! :-) http://...,Delta,NaN,NaN,positive
1,"@united they're not, actually. gate agent was ...",United,chicago,NaN,negative
2,@AmericanAir No worries they called back 4 hrs...,American,"Dallas, Texas",NaN,negative
3,@united thank you. There was one here a few mo...,United,"New York, NY",America/New_York,positive
4,@united Brothers luggage was lost on Copa Airl...,United,"Kearney, Nebraska",Central Time (US & Canada),negative


In [24]:
display(DataFrameSummary(df_tweets).summary())

,text,airline,tweet_location,user_timezone,sentiment
count,10387,10387,6972,6901,10387
unique,10258,6,2429,76,2
top,@AmericanAir thanks,United,"Washington, DC",Eastern Time (US & Canada),negative
freq,5,2822,120,2673,8242
counts,10387,10387,6972,6901,10387
uniques,10258,6,2429,76,2
missing,0,0,3415,3486,0
missing_perc,0%,0%,32.88%,33.56%,0%
types,categorical,categorical,categorical,categorical,bool


In [29]:
my_tok = spacy.load('en_core_web_lg')
TEXT = data.Field(lower=True, tokenize=my_tok)

In [ ]:
os.makedirs(f'{PATH}trn/positive', exist_ok=True)
os.makedirs(f'{PATH}val/positive', exist_ok=True)
os.makedirs(f'{PATH}trn/negative', exist_ok=True)
os.makedirs(f'{PATH}val/negative', exist_ok=True)
os.makedirs(f'{PATH}all/trn', exist_ok=True)
os.makedirs(f'{PATH}all/val', exist_ok=True)
os.makedirs(f'{PATH}models', exist_ok=True)

In [25]:
val_idxs = get_cv_idxs(len(df_tweets))

In [30]:
md = LanguageModelData.from_dataframes(f'{PATH}all/', TEXT, "text", bs=bs, bptt=bptt, min_freq=10)
# LanguageModelData.from_text_files(f'{PATH}all/', TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

TypeError: from_dataframes() missing 2 required positional arguments: 'train_df' and 'val_df'

In [28]:
df_val=[tweets[x] for x in val_idxs]

KeyError: 8954

In [ ]:
spacy_tok = spacy.load('en')

In [6]:
os.makedirs(f'{PATH}trn/yes', exist_ok=True)
os.makedirs(f'{PATH}val/yes', exist_ok=True)
os.makedirs(f'{PATH}trn/no', exist_ok=True)
os.makedirs(f'{PATH}val/no', exist_ok=True)
os.makedirs(f'{PATH}all/trn', exist_ok=True)
os.makedirs(f'{PATH}all/val', exist_ok=True)
os.makedirs(f'{PATH}models', exist_ok=True)

In [7]:
for (i,(_,r)) in enumerate(df_all.iterrows()):
    dset = 'trn' if random.random()>0.1 else 'val'
    open(f'{PATH}all/{dset}/{i}.txt', 'w').write(r['txt'])

In [8]:
for (i,(_,r)) in enumerate(df_mb.iterrows()):
    lbl = 'yes' if r.tweeted else 'no'
    dset = 'trn' if random.random()>0.1 else 'val'
    open(f'{PATH}{dset}/{lbl}/{i}.txt', 'w').write(r['txt'])

In [9]:
from spacy.symbols import ORTH

# install the 'en' model if the next line of code fails by running:
#python -m spacy download en              # default English model (~50MB)
#python -m spacy download en_core_web_md  # larger English model (~1GB)
my_tok = spacy.load('en')

my_tok.tokenizer.add_special_case('<SUMM>', [{ORTH: '<SUMM>'}])
my_tok.tokenizer.add_special_case('<CAT>', [{ORTH: '<CAT>'}])
my_tok.tokenizer.add_special_case('<TITLE>', [{ORTH: '<TITLE>'}])
my_tok.tokenizer.add_special_case('<BR />', [{ORTH: '<BR />'}])
my_tok.tokenizer.add_special_case('<BR>', [{ORTH: '<BR>'}])

def my_spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(x)]

In [10]:
TEXT = data.Field(lower=True, tokenize=my_spacy_tok)
FILES = dict(train='trn', validation='val', test='val')
md = LanguageModelData.from_text_files(f'{PATH}all/', TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [11]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(1271, 14412, 1, 5699790)

In [12]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'the', ',', '.', 'of', '-', 'and', 'a', 'to', 'in', 'we']

In [13]:
' '.join(md.trn_ds[0].text[:150])

'<cat> mathac <summ> in this paper we study the problem of deterministic factorization of sparse polynomials . we show that if $ f \\in \\mathbb{f}[x_{1},x_{2},\\ldots , x_{n}]$ is a polynomial with $ s$ monomials , with individual degrees of its variables bounded by $ d$ , then $ f$ can be deterministically factored in time $ s^{\\mathrm{poly}(d ) \\log n}$. prior to our work , the only efficient factoring algorithms known for this class of polynomials were randomized , and other than for the cases of $ d=1 $ and $ d=2 $ , only exponential time deterministic factoring algorithms were known .    a crucial ingredient in our proof is a quasi - polynomial sparsity bound for factors of sparse polynomials of bounded individual degree . in particular we show if $ f$ is an $ s$-sparse polynomial in $ n$ variables , with individual degrees of its variables'

### Train

In [14]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [15]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
#                dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [16]:
learner.fit(3e-3, 1, wds=1e-6)

epoch      trn_loss   val_loss                                
    0      4.621735   4.502018  



[array([4.50202])]

In [17]:
learner.fit(3e-3, 3, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss                                
    0      4.369812   4.278798  
    1      4.290543   4.169997                                
    2      4.159827   4.094938                                
    3      4.194054   4.091485                                
    4      4.098588   4.012515                                
    5      4.02392    3.963573                                
    6      3.982089   3.947681                                



[array([3.94768])]

In [18]:
learner.save_encoder('adam2_enc')

In [19]:
learner.fit(3e-3, 10, wds=1e-6, cycle_len=5, cycle_save_name='adam3_10')

epoch      trn_loss   val_loss                                
    0      4.093926   4.003164  
    1      4.039814   3.960781                                
    2      3.973589   3.917064                                
    3      3.890041   3.886201                                
    4      3.871082   3.880378                                
    5      4.020988   3.946259                                
    6      3.959853   3.908145                                
    7      3.891331   3.873749                                
    8      3.830333   3.849011                                
    9      3.855924   3.843783                                
    10     3.979082   3.915358                                
    11     3.91581    3.881344                                
    12     3.858311   3.848436                                
    13     3.796537   3.826019                                
    14     3.763215   3.822233                                
    15     3.94726    

[array([3.76542])]

In [20]:
learner.save_encoder('adam3_10_enc')

In [ ]:
learner.fit(3e-3, 8, wds=1e-6, cycle_len=10, cycle_save_name='adam3_5')

epoch      trn_loss   val_loss                                
    0      3.83615    3.837903  
    1      3.821765   3.82772                                 
    2      3.814529   3.819188                                
    3      3.77562    3.80498                                 
    4      3.761399   3.792065                                
    5      3.695456   3.776722                                
    6      3.659784   3.766794                                
    7      3.636959   3.758339                                
    8      3.619205   3.755518                                
 90%|████████▉ | 1139/1271 [03:27<00:24,  5.48it/s, loss=3.61]

In [ ]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=20, cycle_save_name='adam3_20')

In [ ]:
learner.save_encoder('adam3_20_enc')

In [ ]:
learner.save('adam3_20')

### Test

In [ ]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])

In [ ]:
m=learner.model

In [ ]:
s="""<CAT> cscv <SUMM> algorithms that"""

In [ ]:
def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        print(word, end=' ')
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0))

    m[0].bs=bs

In [ ]:
sample_model(m,"<CAT> csni <SUMM> algorithms that")

In [ ]:
sample_model(m,"<CAT> cscv <SUMM> algorithms that")

In [ ]:
sample_model(m,"<CAT> cscv <SUMM> algorithms. <TITLE> on ")

In [ ]:
sample_model(m,"<CAT> csni <SUMM> algorithms. <TITLE> on ")

In [ ]:
sample_model(m,"<CAT> cscv <SUMM> algorithms. <TITLE> towards ")

In [ ]:
sample_model(m,"<CAT> csni <SUMM> algorithms. <TITLE> towards ")

### Sentiment

In [ ]:
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

In [ ]:
class ArxivDataset(torchtext.data.Dataset):
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for label in ['yes', 'no']:
            fnames = glob(os.path.join(path, label, '*.txt'));
            assert fnames, f"can't find 'yes.txt' or 'no.txt' under {path}/{label}"
            for fname in fnames:
                with open(fname, 'r') as f: text = f.readline()
                examples.append(data.Example.fromlist([text, label], fields))
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data',
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)

In [ ]:
ARX_LABEL = data.Field(sequential=False)
splits = ArxivDataset.splits(TEXT, ARX_LABEL, PATH, train='trn', test='val')

In [ ]:
md2 = TextData.from_splits(PATH, splits, bs)

In [ ]:
#            dropout=0.3, dropouti=0.4, wdrop=0.3, dropoute=0.05, dropouth=0.2)

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

def prec_at_6(preds,targs):
    precision, recall, _ = precision_recall_curve(targs==2, preds[:,2])
    print(recall[precision>=0.6][0])
    return recall[precision>=0.6][0]

In [ ]:
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.65, wdrop=0.5, dropoute=0.1, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.clip=25.

In [ ]:
# this notebook has a mess of some things going under 'all/' others not, so a little hack here
!ln -sf ../all/models/adam3_20_enc.h5 {PATH}models/adam3_20_enc.h5
m3.load_encoder(f'adam3_20_enc')
lrs=np.array([1e-4,1e-3,1e-3,1e-2,3e-2])

In [ ]:
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

In [ ]:
m3.fit(lrs, 2, metrics=[accuracy], cycle_len=4, cycle_save_name='imdb2')

In [ ]:
prec_at_6(*m3.predict_with_targs())

In [ ]:
m3.fit(lrs, 4, metrics=[accuracy], cycle_len=2, cycle_save_name='imdb2')

In [ ]:
prec_at_6(*m3.predict_with_targs())